# Fine tuning Roberta Large

## Creating the dataset
We need to prepare a dataset with two categories: *offer letter* and *not offer letter*.

In [1]:
# install dependencies
%pip install pandas scikit-learn --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pathlib import Path
import pandas as pd


def create_dataframe_from_directory(directory: str):
    # Define the paths to your directories
    offer_letters_dir = Path('./datasets')/ directory / 'offer_letters'
    not_offer_letters_dir = Path('./datasets')/ directory / 'not_offer_letters'

    # Initialize lists to store file names and labels
    data = []

    # Process offer letters
    for filename in os.listdir(offer_letters_dir):
        if filename.endswith('.txt'):
            with open(os.path.join(offer_letters_dir, filename), 'r') as file:
                text = file.read()
                data.append({'text': text, 'label': 1})

    # Process not offer letters
    for filename in os.listdir(not_offer_letters_dir):
        if filename.endswith('.txt'):
            with open(os.path.join(not_offer_letters_dir, filename), 'r') as file:
                text = file.read()
                data.append({'text': text, 'label': 0})

    # Create a DataFrame
    return pd.DataFrame(data)

In [3]:
df = create_dataframe_from_directory('training')

# Display the first few rows of the DataFrame
df.head()

,text,label
0,"101 Innovation Way\nSeattle, WA 98101\nPhone: ...",1
1,"August 7, 2024\n\nJane Doe\n123 Maple Street\n...",1
2,Green Earth Technologies\n789 Eco Lane\nSan Fr...,1
3,"City of Springfield\n\nAugust 4, 2024\n\nMicha...",1
4,"Tech Innovations Inc.\n\nAugust 1, 2024\n\nJes...",1


## Splitting the dataset for training and evaluation
Let's use the `train_test_split` function from sklearn to split the dataset.

In [4]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Display the first few rows
train_data.head()


,text,label
8,"Creative Minds Studio\n\nAugust 2, 2024\n\nJoh...",1
16,LEASE RENEWAL AGREEMENT\n\nThis Lease Renewal ...,0
3,"City of Springfield\n\nAugust 4, 2024\n\nMicha...",1
13,"Bright Future Marketing\n\nAugust 1, 2024\n\nE...",0
15,FOR IMMEDIATE RELEASE\nContact:\nEmily Richard...,0


## Fine tuning the model

In [5]:
# install dependencies
%pip install transformers datasets torch torchvision torchaudio --quiet

Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
torch.cuda.is_available()

True

## Tokenize the datasets
let's tokenize the text in our datasets so that we can feed it to our model.

In [7]:
os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset

# Load the tokenizer and model
model_name = "roberta-large-mnli"  # Specialized model for text classification
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True).to('cuda')

# Tokenize the datasets
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

train_dataset = Dataset.from_pandas(train_data).map(tokenize_function, batched=True)
val_dataset = Dataset.from_pandas(val_data).map(tokenize_function, batched=True)


/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you exp

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

## Fine tune the model

In [9]:
# install dependencies
%pip install accelerate --quiet

Note: you may need to restart the kernel to use updated packages.


In [10]:
os.environ["TOKENIZERS_PARALLELISM"] = "True"

In [11]:
from transformers import Trainer, TrainingArguments

# Prepare training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_checkpointing=True,
    num_train_epochs=4,
    weight_decay=0.01,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


## Evaluate the model

In [ ]:
trainer.evaluate()


{'eval_loss': 0.8058374524116516,
 'eval_runtime': 0.298,
 'eval_samples_per_second': 13.421,
 'eval_steps_per_second': 3.355,
 'epoch': 3.0}

## Save the model

In [ ]:
model.save_pretrained("./saved_models/roberta-large")
tokenizer.save_pretrained("./saved_models/roberta-large")

('./saved_models/roberta-large/tokenizer_config.json',
 './saved_models/roberta-large/special_tokens_map.json',
 './saved_models/roberta-large/vocab.json',
 './saved_models/roberta-large/merges.txt',
 './saved_models/roberta-large/added_tokens.json',
 './saved_models/roberta-large/tokenizer.json')

## Check model accuracy

In [ ]:
from sklearn.metrics import accuracy_score

# Load your evaluation dataset

test_dataset = create_dataframe_from_directory("testing")

# Assuming 'features' are in columns except the last one and 'label' is the last column
X_eval = test_dataset.iloc[:, :-1]  # Features (all columns except the last)
y_eval = test_dataset.iloc[:, -1]    # Labels (last column)

# Load your trained DistilBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('./saved_models/roberta-large')
model = AutoModelForSequenceClassification.from_pretrained('./saved_models/roberta-large', num_labels=2)

# Tokenize the evaluation data
inputs = tokenizer(X_eval.iloc[:, 0].tolist(), padding=True, truncation=True, return_tensors='pt')

# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Get the predicted class labels
y_pred = torch.argmax(logits, dim=1).numpy()

# Calculate accuracy
accuracy = accuracy_score(y_eval, y_pred)

print(f'Model Accuracy on Evaluation Dataset: {accuracy * 100:.2f}%')

Model Accuracy on Evaluation Dataset: 40.00%
